# 04 — Visualization Training
## Recreating dashboard-style charts from pipeline data

This notebook builds the same visualizations used in the Streamlit dashboard: speed gauge, battery, interventions, miles-per-intervention, and perception bar charts.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from src.metrics.queries import (
    latest_telemetry,
    interventions_per_vehicle,
    miles_per_intervention,
    perception_summary,
)

### Speed gauge (latest vehicle)

In [ ]:
telemetry = latest_telemetry()
if not telemetry.empty:
    speed = float(telemetry.iloc[0]["current_speed_kmh"])
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=speed,
        title={"text": "Speed (km/h)"},
        gauge={"axis": {"range": [0, 120]}, "threshold": {"line": {"color": "red"}, "value": 65}},
    ))
    fig.update_layout(height=250)
    fig.show()
else:
    print("No telemetry data.")

### Interventions per vehicle (bar)

In [ ]:
interventions = interventions_per_vehicle(hours=24)
if not interventions.empty:
    fig = px.bar(interventions, x="vehicle_id", y="event_count", color="event_type", barmode="group",
                 title="Driving events per vehicle (last 24h)")
    fig.show()
else:
    print("No driving events.")

### Km per intervention (table + bar)

In [ ]:
mpi = miles_per_intervention(hours=24)
if not mpi.empty and "km_per_intervention" in mpi.columns:
    mpi_clean = mpi.dropna(subset=["km_per_intervention"])
    if not mpi_clean.empty:
        fig = px.bar(mpi_clean, x="vehicle_id", y="km_per_intervention", title="Km per intervention (last 24h)")
        fig.show()
    display(mpi)
else:
    print("No miles-per-intervention data.")

### Perception: detections by object class

In [ ]:
perception = perception_summary(hours=24)
if not perception.empty:
    fig = px.bar(perception, x="object_class", y="detection_count", color="vehicle_id", barmode="stack",
                 title="Detections by object class")
    fig.show()
else:
    print("No perception events.")